<a href="https://colab.research.google.com/github/EmmaTopolovec/AlexaControlledLEDs/blob/main/assign1_topolovec_B00879163.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CS 580E Assignment 1
**Emma Topolovec | B00879163**


---

##Part 1 [16 pts]: Numerical Inversion by Binary Search

For this part, you are to write a function named invert that has two required parameters and one optional parameter d. The required parameters are another function f(x) and a y value. Your function should return the x value such that f(x) = y. The function is guaranteed to be non-decreasing, i.e., the derivative is always 0 or positive. If a 3rd argument is given, it will be a tuple (if using Python) representing an interval where the function is guaranteed to be non-decreasing. Otherwise, you may assume that the function is everywhere non-decreasing.

Make sure your function always terminates.

Place your function in a separate cell.

You will then apply your invert function to the two functions below.

In [76]:
# @title Part 1 Function
import math

def invert(f, y, d=None):
  # Initialize low and high
  low = 0
  high = 0
  mid = -1
  prev_mid = -1

  # If d is not None, then d has the low and high
  if d != None:
    low = d[0]
    high = d[1]

    # Error check: if y is not within f(low) and f(high), then invalid input
    if y > f(high) or y < f(low):
      raise Exception('Invalid Input: given y value is not within the given range')

  # Else, loop exponentially until f(low) < y and f(high) > y
  else:
    low = -1
    high = 1
    while f(low) > y or f(high) < y:
      low *= 10
      high *= 10

  # Now, perform binary search
  while low <= high:

    # Set mid
    mid = (high + low) / 2.0

    v = f(mid)

    """
    To get the most precision, we first try to check if the predicted
    inverse yields our target y exactly. If not, we keep looping. If at any
    point, our mid value stays the same, then we can switch over to
    checking if the values are within 1e-15 apart.
    """

    if mid == prev_mid:
      # If f(mid) is equal to y --> mid is x
      if math.isclose(v, y, rel_tol=1e-15):
        return(mid)
    elif v == y:
      return(mid)

    # Update prev_mid
    prev_mid = mid

    # If f(mid) is less than y, ignore lower half
    if v < y:
      low = mid
    # If f(mid) is greater than y, ignore upper half
    elif v > y:
      high = mid

  # Element not found? should never trigger
  return(mid)

**Part 1.1 [8 pts]: Transcendental Functions**

Apply your invert function to this function:

f(x) = x<sup>e</sup> + x

In a separate cell, write a test to make sure that your invert routine is able to invert the function everywhere. We leave it up to you to think about how to design and implement such a test.

In [ ]:
# @title Part 1.1 Tests

test_values = [-19902.3231, -100, -1, -0.000001, 0, .000001, 1, 100, 10993.234, 992910492.123424]

for y in test_values:
  try:
    inverse = invert(lambda x : x**math.e + x, y, (0, 1e9)) # Find inverse - We MUST give the function an interval as it does not exist when x < 0
    predicted_y = inverse**math.e + inverse

    print(f"Inverse of {y}: {inverse} - Loss of precision: {abs(y - predicted_y)}")
  except Exception as e: # No inverse can be found
    print(f"No inverse of {y} - {e}")

**Part 1.2 [8 pts]: Gamma Function**

The gamma function is component of many probability distribution functions and has a wide range of applications in probability, statistics, and physics. The gamma function is defined as an improper integral:

Γ(x) = ∫<sup>∞</sup><sub>0</sub> t<sup>x-1</sup>e<sup>-t</sup>dt

In a new cell, write a test to make that your invert routine is able to invert the Gamma function in the domain (1.5, 16). You may use the gamma function defined in SciPy.

In [ ]:
# @title Part 1.2 Tests

import scipy

test_values = [-19902.3231, -2, 0, 1, 1.5, 2, 2.4532, 12.4532, 15.9999, 16, 100, 110.32]

for y in test_values:
  try:
    inverse = invert(scipy.special.gamma, y, (1.5, 16)) # Find inverse
    predicted_y = scipy.special.gamma(inverse)

    print(f"Inverse of {y}: {inverse} - Loss of precision: {abs(y - predicted_y)}")
  except Exception as e: # No inverse can be found
    print(f"No inverse of {y} - {e}")

---

##Part 2 [7 pts]: Numerical Integration

A large class of continuous functions do not have closed-form solutions for their antiderivatives. In this case, the definite integral over a range can be computed numerically. Write a function integrate that has two parameters: another function f(x) and a tuple representing the range of integration. Your function should return the definite integral of the function f(x), computed over the given range.

Place your function in a separate cell.

In the following cell define the function below, and compute the definite integral numerically using your integrate function.

∫<sub>-1</sub><sup>1</sup>(sin(x)/x)dx

You may use Monte Carlo or other sampling-based methods, as long as the program terminates in a reasonable amount of time. Your function should return the integral value with as high a precision as possible.

In [74]:
# @title Part 2 Function

def integrate(f, d):
  # Trapezoidal Riemann Sums
  integral = 0
  samples = 100000
  x_range = d[1]-d[0]
  trap_width = x_range / float(samples)
  for i in range(samples+1):
    if i == 0 or i == samples: # first or last trapezoid
      integral += f(d[0] + trap_width * i)
    else:
      integral += 2 * f(d[0] + trap_width * i)
  return integral / float(samples)

In [ ]:
# @title Part 2 Test

import math

def f_part2(x):
  if x == 0: # Can't divide by 0
    return(1)
  return(math.sin(x)/x)

print(f"The integral of sin(x)/x from -1 to 1 is {integrate(f_part2, (-1, 1))}")

---

##Part 3 [39 pts]: Gradient Descent

For this part, you will write a gradient descent optimizer and use it to minimize some functions. In the below, we’ll call the function to be optimized f. Write a function opt with three parameters: a function grad(p), a tuple start giving the starting point of the optimization, and the step size step. The function grad(p) should accept a point p as a tuple and returns a tuple giving the gradient of f at p.

Your function opt should return an array of tuples consisting of the sequence of points where the gradient is evaluated, starting from the start point. The last element of the returned array should be the point that minimizes f. The point may be a local mininum.

Note: Make sure your function always terminates.

Place your function in a separate cell. You will then apply your opt function to two functions below.

In [119]:
# @title Part 3 Function

import math

def opt(grad, start, step_size):
  # List to store the sequence of points
  points = [start]

  # Current point
  curr_p = start

  i = 0
  while True:
    i += 1

    # Compute the gradient at the current point
    gradient = grad(curr_p)

    # Update the current point using gradient descent
    curr_p = tuple(p - step_size * g for p, g in zip(curr_p, gradient))

    # If the point is already in the sequence, then we will be stuck in an infinite loop
    # Only do this sometimes to be efficient
    if i % 10000 == 0 and curr_p in points:
      points.append(curr_p)
      return points

    # Add the new point to the sequence
    points.append(curr_p)

    # Check for convergence (if the gradient is close to zero)
    if math.isclose(math.sqrt(gradient[0]**2 + gradient[1]**2), 0, rel_tol=1e-6):
        break

  return points

**Part 3.1 [15 pts]: Three-Hump Camel Function**

The three-hump camel function is a two-dimensional function often used to benchmark optimization methods. It has multiple extrema. The three-hump camel function is defined as:

f(x<sub>1</sub>, x<sub>2</sub>) = 2x<sub>1</sub><sup>2</sup> - 1.05x<sub>1</sub><sup>4</sup> + x<sub>1</sub><sup>6</sup>/6 + x<sub>1</sub>x<sub>2</sub> + x<sub>2</sub><sup>2</sup>

where, -5 <= x<sub>i</sub> <= 5, i = 1,2


1.   Implement the function thcf with a single tuple parameter p, giving the coordinates x1 and x2 to evaluate the function.
2.   Implement the gradient function, thcf_grad that accepts a point and returns the gradient of that point, both as tuples.
3.    In this cell, put in code that checks using a finite differnce that your thcf_grad function is implemented correctly. It should use randomization of the point at which to check the gradient, and allow running for any number of checks. It should be as convincing as possible.
4.    Find the global minimum, possibly using multiple runs of your opt function. Your code should not have a hard-coded starting point.
5.    Generate a surface plot or contour plot of the function, showing the path of the gradient descent that found the global minimum. You can use the matplotlib library.

In [176]:
# @title Part 3.1 Code

import random

# 1 - Three-Hump Camel Function
def thcf(p):
  x1 = p[0]
  x2 = p[1]

  # If x1 or x2 are out of range, put them in the range
  if x1 > 5:
    x1 = 5
  elif x1 < -5:
    x1 = -5
  if x2 > 5:
    x2 = 5
  elif x2 < -5:
    x2 = -5

  return (2 * x1**2 - 1.05 * x1**4 + x1**6 / 6 + x1 * x2 + x2**2)

# 2 - Three-Hump Camel Gradient Function
def thcf_grad(p):
  x1 = p[0]
  x2 = p[1]

  # If x1 or x2 are out of range, put them in the range
  if x1 > 5:
    x1 = 5
  elif x1 < -5:
    x1 = -5
  if x2 > 5:
    x2 = 5
  elif x2 < -5:
    x2 = -5

  dx1 = 4*x1 - 4*1.05*x1**3 + x1**5 + x2
  dx2 = x1 + 2*x2
  return (dx1, dx2)

# 3 - Finite difference to test thcf_grad()
def finite_difference(n=1000, epsilon=1e-9):
  print("Running finite_difference() function to test thcf_grad()")
  # random.seed(0)
  passes = 0
  for test_num in range(n):
    # Generate random point within range
    p = (random.uniform(-5, 5), random.uniform(-5, 5))

    # Find exact gradient
    gradient = thcf_grad(p)

    # Compute finite difference for both dimension
    a1 = (p[0] + epsilon, p[1])
    b1 = (p[0] - epsilon, p[1])
    a2 = (p[0], p[1] + epsilon)
    b2 = (p[0], p[1] - epsilon)

    # Find approximate gradient for both dimensions
    approx_gradient = ( (thcf(a1) - thcf(b1)) / (2 * epsilon),
                        (thcf(a2) - thcf(b2)) / (2 * epsilon) )

    # Check if the actual gradient is close to the approximate
    x1_error = abs((gradient[0] - approx_gradient[0]) / gradient[0])
    x2_error = abs((gradient[1] - approx_gradient[1]) / gradient[1])
    error_allowed = .01 # 1 percent error allowed
    if x1_error < error_allowed and x2_error < error_allowed:
      passes += 1
    else:
      print(f"\n! Gradient check failed for {p}")
      print(f"! Percent Error:({x1_error*100}, {x2_error*100})")
      print(f"! Actual gradient - {gradient}")
      print(f"! Approx gradient - {approx_gradient}")

  print(f"{passes}/{n} tests passed.\n")

finite_difference()

# 4 - Global Minimum of Three-Hump Camel Function
def find_thcf_min():
  print("Running find_thcf_min() function to find the minimum")
  # random.seed(0)
  min_point = None
  min_val = None
  samples = 50
  for i in range(samples):
    # Generate random point to start
    p = (random.uniform(-5, 5), random.uniform(-5, 5))

    # Get min - use random step size
    step_size = random.uniform(0.0001, .05)
    curr_min_point = opt(thcf_grad, p, step_size)[-1]
    curr_min_val = thcf(curr_min_point)

    # Update global minimum if the current minimum is smaller
    if min_val == None or curr_min_val < min_val:
      min_val = curr_min_val
      min_point = curr_min_point

  # Return the minimum minimum
  return(min_point, min_val)

thcf_min = find_thcf_min()
print(f"Global minimum: f{thcf_min[0]} = {thcf_min[1]}")

Running finite_difference() function to test thcf_grad()
1000/1000 tests passed.

Running find_thcf_min() function to find the minimum
Global minimum: f(4.0247393355423205e-163, -9.716580288882752e-163) = 0.0
